# Text Analytics Lab 2: Sequence Labelling

In this lab we will implement an HMM for part-of-speech (POS) tagging, then see how to use a library to run an HMM and a CRF for named entity recognition.

### Outcomes
* Be able to implement the main parts of a supervised HMM.
* Understand what the steps of Viterbi are doing.
* Know how to use a CRF and specify the features it uses.
* Be able to extract named entities using a sequence tagger.

### Overview

The first part of the notebook loads a POS dataset from the NLTK library.
The second part implements and tests an HMM POS tagger.
The third part uses CRFs for the task of named entity recognition.

## Background

### Hidden Markov Model (HMM)

In this lab we will look into Hidden Markov Model (HMM) to model sequential data. HMMs are based on the Markov assumption which states that the present state $z_n$ is sufficient to predict the future $y_{n+1}$ so the past $y_{0:n-1}$ can be forgotten.

Often, the states we are interested in cannot be observed directly -- they are 'hidden'. As we will see in Exercise 2, the part-of-speech (POS) tags are hidden states that we want to predict. We can only observe the words, and have to use them to infer the tags. 
An HMM is specified by the following components:

* A set of $N$ states.

* A transition probability matrix $A$ where each element $a_{ij}$ represents the probability of moving from state $i$ to state $j$, s.t.  $ \sum^{N}_{j=1} a_{ij} = 1$ $ \forall i$

* An emission probability distribution, the probabilities of observations $x_n$ being generated from a state $y_n$

* An initial probability distribution over states. $\pi_n$ is the probability that the Markov chain will start in state $n$. Also, $\sum^{N}_{n=1} \pi_n = 1 $

## Part of Speech (POS) Tagging

Part-of-speech (PoS) tagging extracts syntactic information about words in a sentence, which can then be used to understand how they relate to neighbouring words. The parts of speech are the grammatical categories of words, such as nouns, verbs, and adjectives. Parts of speech are useful features for information extraction, e.g., for labelling named entities, such as people or organisations, which are often proper nouns. A word’s part of speech can even play a role in speech recognition or speech synthesis, e.g., the word 'content' is pronounced CONtent when it is a noun and conTENT when it is an adjective.

PoS tagging assigns a single PoS tag to each token in a text sequence. Therefore, the input to a tagging algorithm is a sequence of tokenised words and the output is a sequence of tags, one per token. Hence, PoS-tagging is a specific case of the more generic NLP task of sequence labelling.
 Many words can have different PoS tags in different contexts, so the goal is to find the correct tag for a particular usage of a word in a sentence. For example, "book" can be a verb ("book that flight") or a noun ("hand me that book"). The challenge of PoS-tagging is to model the context of a token to resolve these ambiguities and choose the correct tag for the context. 

# 1. Preparing the PoS Tagging Data

For PoS tagging, we are going to start with the [Brown corpus](https://www.nltk.org/nltk_data/), which contains many different sources of English text (books, essays, newspaper articles, government documents...) collected and hand-labelled by linguists in 1967.

If you would like to try out PoS tagging in another language, you can uncomment the code below to switch to the [NLTK Indian corpus](https://www.nltk.org/_modules/nltk/corpus/reader/indian.html), which contains datasets for POS tagging in Bangla, Hindi, Marathi and Telugu, and you are welcome to use these alternative datasets for this lab.

In [1]:
import nltk

# If you want to try another language, try Hindi:
# nltk.download('indian')  # the dataset
# from nltk.corpus import indian
# nltk_data = list(indian.tagged_sents('hindi.pos'))

nltk.download('brown')  # download Brown corpus
nltk.download('universal_tagset')   # download the POS tags data
from nltk.corpus import brown
nltk_data = list(brown.tagged_sents(tagset='universal'))

[nltk_data] Downloading package brown to /Users/es1595/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/es1595/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


First we need to split the dataset into training and test sets. Run the following cells to achieve that and to convert the dataset to list format, which will be easier to work with in the following steps.

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

train_set, test_set = train_test_split(
    nltk_data,
    train_size=0.80,  # use 80% as the training data
    test_size=0.20,
    random_state=101
)
print(f'Number of training sentences: {len(train_set)}')
print(f'Number of test sentences: {len(test_set)}')

# Separate the labels from the text
train_toks = []  # each item in the list is a list of tokens in a document
train_tags = []  # each item in the list is a list of corresponding tags
for tagged_sentence in train_set:
    sentence_toks = []
    sentence_tags = []
    for token, tag in tagged_sentence:
        sentence_toks.append(token)
        sentence_tags.append(tag)

    train_toks.append(sentence_toks)
    train_tags.append(sentence_tags)

test_toks = []
test_tags = []
for tagged_sentence in test_set:
    sentence_toks = []
    sentence_tags = []
    for token, tag in tagged_sentence:
        sentence_toks.append(token)
        sentence_tags.append(tag)
    test_toks.append(sentence_toks)
    test_tags.append(sentence_tags)

print(f'Number of training sentences in train_toks: {len(train_toks)}')
print(f'Number of test sentences in test_toks: {len(test_toks)}')

Number of training sentences: 45872
Number of test sentences: 11468
Number of training sentences in train_toks: 45872
Number of test sentences in test_toks: 11468


Let's see how many words the vocabulary has:

In [3]:
# create list of train and test tagged words
print(f'Number of tagged tokens in the training set: {len([ tok for sent in train_toks for tok in sent ])}')
print(f'Number of tagged tokens in the test set: {len([ tok for sent in test_toks for tok in sent ])}')

Number of tagged tokens in the training set: 927092
Number of tagged tokens in the test set: 234100


Let's expore the different types of tags by running the next cell.

In [4]:
unique_tags = {tag for sent in train_tags for tag in sent}
print(f'Number of possible tags: {len(unique_tags)}')
print(f'Possible tags: {unique_tags}')

Number of possible tags: 12
Possible tags: {'ADJ', 'NOUN', 'DET', 'VERB', 'X', 'PRT', 'ADP', '.', 'CONJ', 'ADV', 'NUM', 'PRON'}


**TO-DO 1:** Find out what the tags mean at https://github.com/slavpetrov/universal-pos-tags.

The next cell shows an exampes sentence from the dataset.

In [5]:
print('Sentence example: {}'.format(train_set[3]))

Sentence example: [('many', 'ADJ'), ('of', 'ADP'), ('their', 'DET'), ('gifted', 'ADJ'), ('members', 'NOUN'), ('were', 'VERB'), ('prominent', 'ADJ'), ('in', 'ADP'), ('the', 'DET'), ('Vatican', 'NOUN'), ('as', 'ADP'), ('physicians', 'NOUN'), (',', '.'), ('musicians', 'NOUN'), (',', '.'), ('bankers', 'NOUN'), ('.', '.')]


Let's build a vocabulary and convert each token to its index:

In [6]:
# The text is already tokenized, so we don't use CountVectorizer or other Sklearn tools. Instead, we introduce a class from
# the Gensim library to build a vocabulary:
from gensim.corpora import Dictionary

# Convert the tokens to IDs in a vocabulary ready for input to our models
dictionary = Dictionary(train_toks + test_toks)

train_toks_encoded = [dictionary.doc2idx(sent) for sent in train_toks]
test_toks_encoded = [dictionary.doc2idx(sent) for sent in test_toks]
print(f'Example sentence: {train_toks_encoded[3]}')

V = len(dictionary.values())  # vocabulary
print(f'Size of vocabulary is {V}')

Example sentence: [41, 28, 46, 40, 42, 47, 45, 23, 31, 37, 38, 44, 11, 43, 11, 39, 0]
Size of vocabulary is 56057


Now, we convert the tags in our training and test sets to their indexes in the vocabulary:

In [7]:
# The LabelEncoder maps the tags (e.g., "ADP") to numeric IDs (e.g., 2):
from sklearn.preprocessing import LabelEncoder

# Convert the tags from their names to numbers
tag_encoder = LabelEncoder()
tag_encoder.fit([tag for sentence in train_tags for tag in sentence])
train_tags_encoded = [tag_encoder.transform(sentence) for sentence in train_tags]
test_tags_encoded = [tag_encoder.transform(sentence) for sentence in test_tags]

num_tags = len(tag_encoder.classes_)

In [8]:
print(f"List of tags: {tag_encoder.classes_}")
print(f"Mappings from tags to IDs:")
for tag in tag_encoder.classes_:
    print(f"{tag}: {tag_encoder.transform([tag])[0]}")

List of tags: ['.' 'ADJ' 'ADP' 'ADV' 'CONJ' 'DET' 'NOUN' 'NUM' 'PRON' 'PRT' 'VERB' 'X']
Mappings from tags to IDs:
.: 0
ADJ: 1
ADP: 2
ADV: 3
CONJ: 4
DET: 5
NOUN: 6
NUM: 7
PRON: 8
PRT: 9
VERB: 10
X: 11


# 2 Implementing the HMM

The two main components of HMMs are the transition model and the observation (or emission) model. The transition model estimates $P(tag_{t+1}|tag_t)$, the probability of the next tag given the current tag. For discrete features, such as tokens, the observation model is the same as the naïve Bayes text classifier. It estimates $P(word|tag_t)$, the probability of observing a word given the current tag. This is similar to how naïve Bayes estimates $P(word|class_c)$ for each word that occurs in a document. We can use the same maximum likelihood estimation approach to estimate these probabilities from our training data, by counting how often each word occurs with a given tag.

Let's start by implementing the transition matrix. 

**TO-DO 2:** Count the state (tag) transitions and starting state (tag) occurrences in the training set and store the counts in the `transitions` and `start_states` matrices below. In `transitions`, rows correspond to states at time t-1, the columns to the following state at time t.

In [9]:
from tqdm import tqdm  # progress bars

transitions = np.zeros((num_tags, num_tags))  # transition matrix. Initially, we will fill this with counts of transitions
start_states = np.zeros(num_tags) # start state probabilities, which we will first fill with counts of how often each state occurs at the start of a sequence

for sentence_tags in tqdm(train_tags_encoded):
    for i, tag in enumerate(sentence_tags):
        if i==0:
            start_states[tag] += 1
            continue
        ### WRITE YOUR OWN CODE HERE
        transitions[sentence_tags[i-1], tag] += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 45872/45872 [00:00<00:00, 75703.23it/s]


**TO-DO 3:** Normalise the transition and start state counts to estimate the conditional probabilities in the transition matrix $A$ and starting state probabilities $\pi$.

In [10]:
### WRITE YOUR CODE HERE
transitions /= np.sum(transitions, 1)[:, None]
start_states /= np.sum(start_states)

Now let's compute the emission/observation model.

**TO-DO 4:** Count the number of occurrences of each word type given each tag.

In [11]:
observations = np.zeros((num_tags, V))  # We will first fill this with counts of words given tags

for i, sentence_toks in tqdm(enumerate(train_toks_encoded)):
    sentence_tags = train_tags_encoded[i]
    for j, tok in enumerate(sentence_toks):
        tag = sentence_tags[j]
        # WRITE YOUR OWN CODE HERE
        observations[tag, tok] += 1

45872it [00:00, 59683.48it/s]


**TO-DO 5:** Normalise the observation counts to obtain the observation probabilities.

In [12]:
#WRITE YOUR OWN CODE HERE
observations /= np.sum(observations, 1)[:, None]

To predict the most likely sequence of tags, we use the Viterbi algorithm, defined in the next cell.

**TO-DO 6:** Compare this implementation with the slides or textbook and try to understand how it works.


In [13]:
def viterbi(observed_seq, num_tags, start_probs, transition_probs, observation_probs):
    eps = 1e-7

    num_obs = len(observed_seq)

    # Initialise the V and backpointers
    V = np.zeros((num_obs, num_tags))
    backpointer = np.zeros((num_obs, num_tags))

    # For the first data point in the sequence:
    V[0, :] = start_probs * observation_probs[:, observed_seq[0]]

    # Run Viterbi forward for t > 0
    for t in range(1, num_obs):

        for state in range(num_tags):
            # probabilities for all the sequences leading to this state at time t
            seq_prob = V[t-1, :] * transition_probs[:, state]

            # Choose the most likely sequence
            max_seq_prob = np.max(seq_prob)
            best_previous_state = np.argmax(seq_prob)

            # Calculate the probability of the most likely sequence leading to this state at time t, including the current observation.
            # Add eps to help with numerical issues.
            V[t, state] = (max_seq_prob + eps) * (observation_probs[state, observed_seq[t]] + eps)

            backpointer[t, state] = best_previous_state

    t = num_obs - 1

    # Initialise the sequence of predicted states
    state_seq = np.zeros(num_obs, dtype=int)

    # Get the most likely final state:
    state_seq[t] = np.argmax(V[t, :])

    # Backtrack until the first observation
    for t in range(len(observed_seq)-1, 0, -1):
        state_seq[t-1] = backpointer[t, state_seq[t]]

    return state_seq

**TO-DO 7:** Use the viterbi function to estimate the most likely sequence of states on the test set.

In [14]:
predictions = []
for sentence in tqdm(test_toks_encoded):
    # WRITE YOUR OWN CODE HERE
    predictions.append(viterbi(sentence, num_tags, start_states, transitions, observations))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 11468/11468 [00:30<00:00, 372.17it/s]


In [15]:
# Viterbi returns a sequence of tag IDs, rather than the actual tags. Now, convert the sequence of tag IDs to tag names:
predicted_tags = []
for sequence in tqdm(predictions):
    predicted_tags.append(tag_encoder.inverse_transform(sequence))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 11468/11468 [00:01<00:00, 9282.96it/s]


**TO-DO 8:** Run the code below to print some example predictions. Find some examples where the HMM makes an incorrect prediction. Can you understand why it makes that error? You can use the observation distributions and transition matrix to investigate further. 

In [16]:
# print some examples:
examples = [2, 334, 4983, 2389]
for eg in examples:
    print(f'Tokens:      {test_toks[eg]}')
    print(f'Gold tag:    {test_tags[eg]}')
    print(f'Predictions: {predicted_tags[eg]}')

Tokens:      ['``', 'My', 'God', ',', "I'm", 'shot', "''", '!', '!']
Gold tag:    ['.', 'DET', 'NOUN', '.', 'PRT', 'VERB', '.', '.', '.']
Predictions: ['.' 'DET' 'NOUN' '.' 'PRT' 'NOUN' '.' '.' '.']
Tokens:      ['She', 'thought', 'she', 'was', 'bigger', 'than', 'we', 'are', 'because', 'she', 'came', 'from', 'Torino', "''", '.']
Gold tag:    ['PRON', 'VERB', 'PRON', 'VERB', 'ADJ', 'ADP', 'PRON', 'VERB', 'ADP', 'PRON', 'VERB', 'ADP', 'NOUN', '.', '.']
Predictions: ['PRON' 'VERB' 'PRON' 'VERB' 'ADJ' 'ADP' 'PRON' 'VERB' 'ADP' 'PRON' 'VERB'
 'ADP' 'NOUN' '.' '.']
Tokens:      ['Meanwhile', ',', 'I', 'reloaded', 'my', 'gun', ',', 'as', 'the', 'other', 'men', 'were', 'doing', '.']
Gold tag:    ['ADV', '.', 'PRON', 'VERB', 'DET', 'NOUN', '.', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'VERB', '.']
Predictions: ['ADV' '.' 'PRON' 'VERB' 'DET' 'NOUN' '.' 'ADP' 'DET' 'ADJ' 'NOUN' 'VERB'
 'VERB' '.']
Tokens:      ['The', 'difficulty', 'was', 'that', 'each', 'day', 'seemed', 'to', 'produce', 'its', 'quot

Let's get an overall picture of the HMM's performance by computing accuracy:

In [17]:
# compute accuracy

from sklearn.metrics import accuracy_score

all_predictions = [tag for sentence in predictions for tag in sentence]
all_targets = [tag for sentence in test_tags_encoded for tag in sentence]

acc = accuracy_score(all_targets, all_predictions)
print(f'Accuracy = {acc}')

Accuracy = 0.9436992738146092


# 3. Named Entity Recognition (NER) with CRF

Named entity recognition is the task of identifying mentions of entities in text, such as people, locations, organisations and times. It is usually modelled as a sequence labelling task, where the goal is to label tokens as either a particular entity type or as not a named entity. However, many entities span more than one token, such as the person "Ada Lovelace" or the organisation "University of Bristol". To show where these spans start and end, we therefore in addition to tagging the entity type, we also tag each token as 'beginning' (at the start of an entity span), 'inside' (continuing an entity span), or 'outside' (not part of an entity). Hence, the complete set of tags include "O" for 'outside', along with "B-Person", "I-Person", "B-Organisation", "I-Organisation", etc., for each entity type. 

Let's load some NER data consisting of English news articles.

In [18]:
from datasets import load_dataset

cache_dir = "./data_cache"

# The data is already divided into training and test sets.
# Load the training set:
train_dataset = load_dataset(
    "conll2003",
    split="train",
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

Training dataset with 14041 instances loaded


In [19]:
# Load the test set:
test_dataset = load_dataset(
    "conll2003",
    split="test",
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset)} instances loaded")

Test dataset with 3453 instances loaded


Let's take a look at one of the instances in the training set:

In [20]:
train_dataset[0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

As well as NER tags, the dataset includes PoS tags and Chunk tags, which identify the grammatical phrases in the sentence.

The tags are all stored by their indexes, rather than as the tags themselves. The mapping of the PoS tags to their indexes is:

```
{'"': 0, "''": 1, '#': 2, '$': 3, '(': 4, ')': 5, ',': 6, '.': 7, ':': 8, '``': 9, 'CC': 10, 'CD': 11, 'DT': 12,
 'EX': 13, 'FW': 14, 'IN': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'LS': 19, 'MD': 20, 'NN': 21, 'NNP': 22, 'NNPS': 23,
 'NNS': 24, 'NN|SYM': 25, 'PDT': 26, 'POS': 27, 'PRP': 28, 'PRP$': 29, 'RB': 30, 'RBR': 31, 'RBS': 32, 'RP': 33,
 'SYM': 34, 'TO': 35, 'UH': 36, 'VB': 37, 'VBD': 38, 'VBG': 39, 'VBN': 40, 'VBP': 41, 'VBZ': 42, 'WDT': 43,
 'WP': 44, 'WP$': 45, 'WRB': 46}
 ```
 
The mapping from indexes to NER tags to indexes is:

```
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
```
 

In [21]:
ner_tag_mapping = {0: 'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5:'B-LOC', 6:'I-LOC', 7:'B-MISC', 8:'I-MISC'}

Let's put the NER data in the right format for NLTK's CRFTagger class. The CRFTagger requires that the data is a sequence of tuples, where each tuple contains the input token and the output tag. 

In [22]:
train_set = [list(zip(s['tokens'], [ner_tag_mapping[tok] for tok in s['ner_tags']])) for s in train_dataset][:-1]
test_set = [list(zip(s['tokens'], [ner_tag_mapping[tok] for tok in s['ner_tags']])) for s in test_dataset][:-1]
test_tokens = [s['tokens'] for s in test_dataset][:-1]
test_tags = [[ner_tag_mapping[tok] for tok in s['ner_tags']] for s in test_dataset][:-1]

Now, let's train a CRF tagger on our training set. The method you need to use from NLTK is the [train method of the conditional random field (CRF)](https://www.nltk.org/_modules/nltk/tag/crf.html). You need to call the constructor with default arguments, then the train() function.

**TO-DO 9:** Write a function to train and return a CRF named entity recogniser.

NOTE: for this step, you need to have the package pycrfsuite installed. If you encounter an error saying that this module cannot be found, run ``conda install python-crfsuite``. 

In [23]:
import nltk

# Train a CRF NER tagger
def train_CRF_NER_tagger(train_set):
    ### WRITE YOUR OWN CODE HERE
    tagger = nltk.tag.CRFTagger()
    tagger.train(train_set, 'model.crf.tagger')
    return tagger  # return the trained model

tagger = train_CRF_NER_tagger(train_set)

Get some predictions from the tagger:

In [24]:
predicted_tags = tagger.tag_sents(test_tokens)

Let's see how well the tagger is performing. In NER, we usually evaluate performance by finding **correctly matched entities, rather than correctly tagged tokens**. Only an exact entity match counts as correct. Therefore, to compute precision, recall and F1 score, we need to compute true positives, false positives and false negatives by looking for the predicted entity spans and the gold-labelled entity spans in the test set.

The code below contains a function that extract a list of spans from the tagged sentences. The next function calls extract_spans() and computes the precision, recall and f1 scores. However, the function is incomplete.

Run the cal_span_level_F1() function below to compute span-level F1 scores for the predictions. Have a look at the results. Which types of entity are being recognised well and which are very poor?

In [25]:
def extract_spans(tagged_sents):
    """
    Extract a list of tagged spans for each named entity type, 
    where each span is represented by a tuple containing the 
    start token and end token indexes.
    
    returns: a dictionary containing a list of spans for each entity type.
    """
    spans = {}
        
    for sidx, sent in enumerate(tagged_sents):
        start = -1
        entity_type = None
        for i, (tok, lab) in enumerate(sent):
            if 'B-' in lab:
                start = i
                end = i + 1
                entity_type = lab[2:]
            elif 'I-' in lab:
                end = i + 1
            elif lab == 'O' and start >= 0:
                
                if entity_type not in spans:
                    spans[entity_type] = []
                
                spans[entity_type].append((start, end, sidx))
                start = -1      
        # Sometimes an I-token is the last token in the sentence, so we still have to add the span to the list
        if start >= 0:    
            if entity_type not in spans:
                spans[entity_type] = []
                
            spans[entity_type].append((start, end, sidx))
                
    return spans


def cal_span_level_f1(test_sents, test_sents_with_pred):
    # get a list of spans from the test set labels
    gold_spans = extract_spans(test_sents)

    # get a list of spans predicted by our tagger
    pred_spans = extract_spans(test_sents_with_pred)
    
    # compute the metrics for each class:
    f1_per_class = []
    
    ne_types = gold_spans.keys()  # get the list of named entity types (not the tags)
    
    for ne_type in ne_types:
        # compute the confusion matrix
        true_pos = 0
        false_pos = 0
        
        for span in pred_spans[ne_type]:
            if span in gold_spans[ne_type]:
                true_pos += 1
            else:
                false_pos += 1
                
        false_neg = 0
        for span in gold_spans[ne_type]:
            if span not in pred_spans[ne_type]:
                false_neg += 1
                
        if true_pos + false_pos == 0:
            precision = 0
        else:
            precision = true_pos / float(true_pos + false_pos)
            
        if true_pos + false_neg == 0:
            recall = 0
        else:
            recall = true_pos / float(true_pos + false_neg)
        
        if precision + recall == 0:
            f1 = 0
        else:
            f1 = 2 * precision * recall / (precision + recall)
            
        f1_per_class.append(f1)
        print(f'F1 score for class {ne_type} = {f1}')
        
    print(f'Macro-average f1 score = {np.mean(f1_per_class)}')

cal_span_level_f1(test_set, predicted_tags)

F1 score for class LOC = 0.7970501474926254
F1 score for class PER = 0.7678398450113012
F1 score for class MISC = 0.6956521739130435
F1 score for class ORG = 0.6519795657726692
Macro-average f1 score = 0.7281304330474099


We can try to help the CRF tagger by adding some more features -- the great thing about the CRF model is that we can easily add in any features that we think may be helpful, and these do not need to be conditionally independent of one another.

In the CRFTagger class, the ```_get_features()``` method extracts the features from the tokens. We can overwrite this method with our own version that provides additional features.

**TO-DO 10:** Add in the previous and next works as features. Be careful with the start and end of the sequence where there is no previous or next word.

In [26]:
import re, unicodedata

class CustomCRFTagger(nltk.tag.CRFTagger):
    _current_tokens = None
    
    def _get_features(self, tokens, idx):
            """
            Extract basic features about this word including
                - Current word
                - is it capitalized?
                - Does it have punctuation?
                - Does it have a number?
                - Suffixes up to length 3

            Note that : we might include feature over previous word, next word etc.

            :return: a list which contains the features
            :rtype: list(str)
            """
            token = tokens[idx]

            feature_list = []

            if not token:
                return feature_list

            # Capitalization
            if token[0].isupper():
                feature_list.append("CAPITALIZATION")

            # Number
            if re.search(self._pattern, token) is not None:
                feature_list.append("HAS_NUM")

            # Punctuation
            punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
            if all(unicodedata.category(x) in punc_cat for x in token):
                feature_list.append("PUNCTUATION")

            # Suffix up to length 3
            if len(token) > 1:
                feature_list.append("SUF_" + token[-1:])
            if len(token) > 2:
                feature_list.append("SUF_" + token[-2:])
            if len(token) > 3:
                feature_list.append("SUF_" + token[-3:])

                
            # Current word
            feature_list.append("WORD_" + token)
            
            ### WRITE YOUR OWN CODE HERE ###
            if idx > 0:
                feature_list.append("PREVWORD_" + tokens[idx-1])
            if idx < len(tokens)-1:
                feature_list.append("NEXTWORD_" + tokens[idx+1])
                
            ####

            return feature_list
                

**TO-DO 11:** Train your custom CRF tagger with your ``_get_features`` method, then test it below. How does it compare to the default tagger? Why do you think adding the new features change the performance in this way? 

The results show how important it is understand your choice of features.

In [27]:
# Train a CRF NER tagger
def train_CustomCRF_NER_tagger(train_set):
    ### WRITE YOUR OWN CODE HERE
    tagger = CustomCRFTagger()
    tagger.train(train_set, 'model.crf.tagger')
    return tagger  # return the trained model

tagger = train_CustomCRF_NER_tagger(train_set)

In [28]:
predicted_tags = tagger.tag_sents(test_tokens)
cal_span_level_f1(test_set, predicted_tags)

F1 score for class LOC = 0.8232400860743928
F1 score for class PER = 0.8225108225108225
F1 score for class MISC = 0.7376425855513309
F1 score for class ORG = 0.7019910083493899
Macro-average f1 score = 0.771346125621484


Part-of-speech tags often provide useful information for identifying entities, e.g., by identifying nouns that may be part of a name. Let's see if they help when dealing with the CoNLL 2003 dataset...

We can use the PoS tagger from NLTK to tag a sentence, as shown in the following cell:

In [33]:
# Download the package for PoS tagging
nltk.download('averaged_perceptron_tagger')

# Some arbitrary token sequence to see how it works...
example_sentence = ["PoS", "tags", "often", "provide", "useful", "information", "for", "identifying", "entities"]

# Tag the sentence...
pos_tagged_tokens = nltk.pos_tag(example_sentence)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/es1595/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [37]:
pos_tagged_tokens  # look at the results

[('PoS', 'NNP'),
 ('tags', 'NNS'),
 ('often', 'RB'),
 ('provide', 'VBP'),
 ('useful', 'JJ'),
 ('information', 'NN'),
 ('for', 'IN'),
 ('identifying', 'VBG'),
 ('entities', 'NNS')]

**TO-DO 12:** Complete the code below to define another custom CRF tagger that also include PoS tags as features. Then, run the final cells to train and test the CRF tagger with PoS tags. How do the results compare with the previous versions?

In [34]:
# *** Improve the CRF NER tagger using parts of speech (see lab 5) as additional features.
class CRFTaggerWithPOS(CustomCRFTagger):
    _current_tokens = None
    
    def _get_features(self, tokens, index):
        """
        Extract the features for a token and append the POS tag as an additional feature.
        """
        basic_features = super()._get_features(tokens, index)
        
        # Get the pos tags for the current sentence and save it
        if tokens != self._current_tokens:
            self._pos_tagged_tokens = nltk.pos_tag(tokens)
            self._current_tokens = tokens
            
            
        ### WRITE YOUR OWN CODE HERE
        basic_features.append(self._pos_tagged_tokens[index][1])
        ###
        
        return basic_features

In [35]:
# Train a CRF NER tagger
def train_CRF_NER_tagger_with_POS(train_set):
    ### WRITE YOUR OWN CODE HERE
    tagger = CRFTaggerWithPOS()
    tagger.train(train_set, 'model.crf.tagger')
    return tagger  # return the trained model

tagger = train_CRF_NER_tagger_with_POS(train_set)

In [36]:
predicted_tags = tagger.tag_sents(test_tokens)
cal_span_level_f1(test_set, predicted_tags)

F1 score for class LOC = 0.8319070904645477
F1 score for class PER = 0.8311530270763615
F1 score for class MISC = 0.751145038167939
F1 score for class ORG = 0.7028227085315573
Macro-average f1 score = 0.7792569660601014
